In [1]:
import cv2
import numpy as np
from keras.models import load_model
from keras.preprocessing import image
from string import ascii_uppercase
import matplotlib.image as mpimg

Using TensorFlow backend.


In [2]:
# Function when trackbar is moved
def nothing(x):
    pass

# Define Model

In [3]:
from keras.models import Sequential 
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization

def from_paper(input_shape=(100,100,1),nclass=24):
    model = Sequential()
    
    model.add(Conv2D(filters=16,
                     input_shape=input_shape,
                     kernel_size=(3, 3),
                     activation='relu',
                     padding='same'))
    
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Conv2D(filters=32,
                     kernel_size=(3, 3),
                     activation='relu',
                     padding='same'))
    
    model.add(MaxPooling2D(pool_size=(5, 5)))
    
    model.add(Conv2D(filters=64,
                     kernel_size=(3, 3),
                     activation='relu',
                     padding='same'))
    
    model.add(MaxPooling2D(pool_size=(5, 5)))
    
    model.add(Dropout(0.2))
    
    model.add(Flatten())
    
    model.add(Dense(1024, activation='relu'))
    
    model.add(Dense(nclass, activation='softmax'))
    
    model.summary()
    
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam', metrics=['accuracy'])
    
    return model

In [4]:
model = from_paper((100,100,1), 24)

model.load_weights('from_paper_model_weight.h5')

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 100, 100, 16)      160       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 50, 50, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 50, 50, 32)        4640      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 10, 10, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 10, 10, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 2, 2, 64)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 2, 2, 64)         

# Live Prediction

In [5]:
def LivePredict():
    cap = cv2.VideoCapture(0)

    # Creating a window
    cv2.namedWindow('result')
    cv2.resizeWindow('result', 600,600)

    # Creating track bar
    cv2.createTrackbar('t1', 'result',80,300,nothing)
    cv2.createTrackbar('t2', 'result',160,300,nothing)

    # Loading model and mapping alphabets
#     model = load_model('model_paper_edge_tembusu_good_standard_40epoch.h5')
    alphabet = [c for c in ascii_uppercase if c not in "JZ"]

    # Loop until user press ESC key
    old_alphabet = '-'
    old_prob = '-'
    while(1):

        # Take video, crop and then resize the frame
        _, frame = cap.read()
        frame = frame[0:720, 0:720]
        frame = cv2.resize(frame,(100,100))

        # Get info from track bar and appy to result
        t1 = cv2.getTrackbarPos('t1','result')
        t2 = cv2.getTrackbarPos('t2','result')

        # Pre-processing through blurring and edge detection
        blur_img = cv2.GaussianBlur(frame,(3,3),0)

        edge = cv2.Canny(blur_img,t1,t2)

        # Predicting
        img_tensor = image.img_to_array(edge)
        img_tensor = np.expand_dims(img_tensor, axis=0)
        img_tensor /= 255.      

        predicted = model.predict(img_tensor, verbose=0)
        pred_alphabet = alphabet[np.argmax(predicted[0])]
        pred_prob = predicted[0][np.argmax(predicted[0])]
        
        if pred_prob > 0.75:
            old_alphabet = pred_alphabet
            old_prob = pred_prob
        else:
            old_alphabet = '-'
            old_prob = '-'

        edge = cv2.resize(edge,(600,600))
        flipped = cv2.flip(edge, 1)

        # Presenting result in the created window
        res = cv2.putText(img=flipped, text='Predicted: '+old_alphabet+', P: '+str(old_prob),
                          org=(20,550),
                          fontFace=cv2.FONT_HERSHEY_COMPLEX,
                          fontScale=1,
                          color=(255,255,255),
                          thickness=2)

        cv2.imshow('result',res)

        # Break
        k = cv2.waitKey(5) & 0xFF
        if k == 27:
            break

    # Destroy created window
    cv2.destroyAllWindows()

In [6]:
LivePredict()